In [1]:
from music21 import *
import pickle, time, os
from sklearn.externals import joblib

In [2]:
A = 'kern_files/Zmo/'
# A = corpus.getComposer('bach') #ignore this for now


#these match the key or mode of the song with the appropriate label
tonal_major_dict = {(0, 0): 'CM', (1, 7): 'GM', (2, 2): 'DM', (3, 9): 'AM', (4, 4): 'EM', (5, 11): 'BM', (6, 6): 'F♯M',
               (-1, 5): 'FM', (-2, 10): 'B♭M', (-3, 3): 'E♭M', (-4, 8): 'A♭M', (-5, 1): 'D♭M', (-6, 6): 'G♭M'}
tonal_minor_dict = {(0, 9): 'am', (1, 4): 'em', (2, 11): 'bm', (3, 6): 'f♯m', (4, 1): 'c♯m', (5, 8): 'g♯m',
                    (6, 3): 'd♯m', (-1, 2): 'dm', (-2, 7): 'gm', (-3, 0): 'cm', (-4, 5): 'fm', (-5, 10): 'b♭m',
                    (-6, 3): 'e♭m'}
modal_major = [('f', 5), ('n', 0), ('f', 0), ('n', 7), ('f', 10), ('n', 5),
               (-1, 5), (0, 0), (-1, 0), (0, 7), (-1, 10), (0, 5)]
modal_minor = [('n', 2), ('n', 9), ('f', 2), ('f', 7), ('n', 4),
               (0, 2), (0, 9), (-1, 2), (-1, 7), (0, 4)]

In [3]:
# this function converts .krn or .xml files into the format we need
def data_converter(unparsed, modality, corpus_name):
    pitch_frequency_data = []
    all_parsed = []
    all_keys = []
    all_chords = []

    start = time.time()
    if type(unparsed) is str:

        for root, dirs, files in os.walk(unparsed):
            for file_name, x in zip(files, range(len(files))):
#                 print(file_name, '...', x + 1, 'of', len(files)+1, '...')
                if file_name.endswith('.krn'):
                    #                     print('it is a krn')
                    path = os.path.join(root, file_name)
                    st = time.time()
                    print('parsing', file_name)
                    all_parsed.append(converter.parse(path))
#                     parser(path)
                    fin = time.time()
#                     print('finished', len(all_chords), 'of', len(unparsed), 'in', fin - st, 'seconds')
    else:
        for x in unparsed:
            st = time.time()
#             print('parsing', x)
            all_parsed.append(converter.parse(x))
#             parser(x)
            fin = time.time()
#             print('finished', len(all_chords), 'of', len(unparsed), 'in', fin - st, 'seconds')
    print('entire process took:', time.time() - start, 'seconds')
    
    def parser(parsed_songs):
        for x in all_parsed:
            if type(x) is stream.Score:
    #             print('we have a score!')
                song_chords = []
                crd = x.chordify().flat.getElementsByClass(chord.Chord)
                if len(x.flat.getElementsByClass(key.KeySignature)) > 0:
                    key_sig = x.flat.getElementsByClass(key.KeySignature)[0].sharps
                    bass_note = crd[-1].bass().pitchClass
                    modal_type = []
                    if modality == 'tonal':
                        for c in tonal_major_dict.keys():
                            modal_type.append(c)
                        for c in tonal_minor_dict.keys():
                            modal_type.append(c)
                            #                 modal_type.append(c for c in tonal_minor_dict.keys())
                    elif modality == 'modal':
                        for c in modal_major:
                            modal_type.append(c)
                        for c in modal_minor:
                            modal_type.append(c)
                            #             print(modal_type, (key_sig, bass_note))
                    if (key_sig, bass_note) in modal_type:
                        print((key_sig, bass_note), 'is in there')
                        for c in crd:
                            song_chord = []
                            note_bass = (c.bass().pitchClass - bass_note) % 12
                            song_chord.append(note_bass)
                            song_chord.append(sorted(set([(n.pitchClass - bass_note - note_bass) % 12 for n in c])))
                            song_chords.append(song_chord)
                        all_chords.append(song_chords)
                        all_keys.append((key_sig, bass_note))
                    # this is for pitch class frequency data
                        all_notes = []
                        note_data = {}
                        song_percents = []
                        note_counter = 0
                        for xx in crd:
                            for yy in xx:
                                all_notes.append((yy.pitchClass - bass_note) % 12)
                                note_counter += 1
                        for xx in range(12):
                            note_data[xx] = 0
                        for xx in all_notes:
                            note_data[xx] += 1
                        for xx in range(12):
                            song_percents.append(note_data[xx] / len(all_notes) * 100)
                        pitch_frequency_data.append(song_percents)
                        print(len(pitch_frequency_data), len(all_keys))
                        print((key_sig, bass_note))
                    #                 all_corpus_labels.append((the_key[0].sharps, bass_note))

            elif type(x) is stream.Opus:
                print('we have an Opus!')
                for y in x:
                    song_chords = []
                    crd = y.chordify().flat.getElementsByClass(chord.Chord)
                    if len(x.flat.getElementsByClass(key.KeySignature)) > 0:
                        key_sig = y.flat.getElementsByClass(key.KeySignature)[0].sharps
                        bass_note = crd[-1].bass().pitchClass
                        modal_type = []
                        if modality == 'tonal':
                            for c in tonal_major_dict.keys():
                                modal_type.append(c)
                            for c in tonal_minor_dict.keys():
                                modal_type.append(c)
                                #                     modal_type.append(c for c in tonal_minor_dict.keys())
                        elif modality == 'modal':
                            for c in modal_major:
                                modal_type.append(c)
                            for c in modal_minor:
                                modal_type.append(c)
                        if (key_sig, bass_note) in modal_type:
                            print((key_sig, bass_note), 'is in there')
                            for c in crd:
                                song_chord = []
                                note_bass = (c.bass().pitchClass - bass_note) % 12
                                song_chord.append(note_bass)
                                song_chord.append(sorted(set([(n.pitchClass - bass_note - note_bass) % 12 for n in c])))
                                song_chords.append(song_chord)
                            all_chords.append(song_chords)
                            all_keys.append((key_sig, bass_note))
                        # this is for pitch class frequency data
                            all_notes = []
                            note_data = {}
                            song_percents = []
                            note_counter = 0
                            for xx in crd:
                                for yy in xx:
                                    all_notes.append((yy.pitchClass - bass_note) % 12)
                                    note_counter += 1
                            for xx in range(12):
                                note_data[xx] = 0
                            for xx in all_notes:
                                note_data[xx] += 1
                            for xx in range(12):
                                song_percents.append(note_data[xx] / len(all_notes) * 100)
                            pitch_frequency_data.append(song_percents)
                            print(len(pitch_frequency_data), len(all_keys))


    parser(all_parsed)
    joblib.dump((all_chords, all_keys), 'pickles/%s_continuo.pkl' % corpus_name, compress=9)
    joblib.dump((pitch_frequency_data, all_keys), 'pickles/%s_notes.pkl' % corpus_name, compress=9)

    print('ALL DONE!!! :) :) :)')

In [4]:
# data_converter(unparsed, modality, corpus_name)
data_converter(A, 'modal', 'TEST')

parsing Jos2106-Magnus_es_tu,_domine.krn
parsing Jos9904-Proch_dolor.krn
parsing Jos1509-Confitemini_Domino.krn
parsing Jos1908-In_principio_erat_verbum.krn
parsing Jos2006-Missus_est_Gabriel.krn
parsing Ort2005-Lamentatio_Jeremie_prophete.krn


KeyboardInterrupt: 

In [5]:
TEST_notes = joblib.load('pickles/Zso_notes.pkl')
TEST_chords = joblib.load('pickles/Zso_continuo.pkl')

In [6]:
TEST_notes[0][0]

[14.749262536873156,
 0.0,
 17.10914454277286,
 15.191740412979351,
 0.0,
 12.831858407079647,
 0.0,
 19.911504424778762,
 2.949852507374631,
 4.277286135693215,
 12.094395280235988,
 0.8849557522123894]